---
title: "Worked example: Thornton Bank"
author: "Kevin Duffy"
bibliography: ../bibliography.bib
---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/triaduct/offshoregeotechnics/blob/main/notes/9-1_thornton-bank-example.ipynb){target="_blank"} <nbsp>

::: {.callout-note}
This is a worked example based on the geometries and loads provided by @peireGravityBaseFoundations2009. Soil properties typical of sand have been assumed. 
:::

# Background 
Thornton Bank is a wind farm 30km off the Belgian coast, in water depths of up to 30m. The first phase of construction was completed in 2009, comprising six 5 MW turbines all founded on gravity base foundations.

::: {#fig-thornton-bank layout-ncol=2}

![[(source)](https://www.maritimejournal.com/concrete-gravity-foundations-the-future-for-windfarms/508526.article)](pics/shfo/thornton-bank.png)

![](pics/shfo/thornton-bank-location.png){width=70%}

Thornton Bank
:::

The first soil layer consisted of ten metres of medium dense coarse sand. This was followed by ten-metre-thick layer of stiff clay and a thirteen-metre-thick layer of dense to very dense sand. A monopile design was considered for the site, although driveability and cost concerns meant that a gravity base foundation was chosen.

An example of one of the foundations is given in @fig-thornton-bank-structure.

![Design of the gravity base foundation at Thornton Bank](pics/shfo/thornton-bank-structure.png){#fig-thornton-bank-structure width=50%}


# Brief
You've been asked to calculate the stability of the foundation in @fig-thornton-bank-structure as per ISO 19901-4:2016. Consider the foundation stability under ambient loading conditions:

1. Calculate the (V, H, M) contributions
2. Calculate the vertical stability of the existing design, including eccentric loading
3. Calculate the resistance to sliding 
4. Are these results satisfactory? What refinements could you make to improve the design?

In doing so, use the following assumptions:

* The foundation is directly on the seabed, with no embedment or scour protection
* Wind and waves act in the same direction and are considered as representative point loads.

Some relevant properties are given in the table below: 

| **Superstructure**            |            |   | **Soil properties**              |                 |
|-------------------------------|------------|---|----------------------------------|-----------------|
| Rotor diameter                | 126 m      |   |  Friction angle $\phi'$          |     32°         |
| Nacelle height above sealevel | 90 m       |   |  Saturated unit weight $\gamma$  |     20 kN/m<sup>3</sup>    |
| Nacelle-rotor weight          | 4.1 MN     |   |  Sliding coefficient $\delta$    |     29°         |
| Tower weight                  | 3 MN       |   |  Material factor $\gamma_m$      |     1.25        |
|                               |            |   |                                  |                 |
| **Substructure**              |            |   | ***To make your life easier***     |                 |
|     Base diameter             | 23.5m      |   |     $s$                          |     77.3        |
|     Concrete weight           | 30 MN      |   |     $K_{\gamma}$                 |     0.7        |
|     Ballast weight            | 38 MN      |   |     $K_q$                        |     1.3        |
|     Buoyant volume            | 2965 m<sup>3</sup>    |   |                       |                 |
|                               |            |   |                                  |                 |
| **Ambient loads**             |            |   |                                  |                 |
|     Wind thrust               | 2.03 MN    |   |                                  |                 |
|     Wind arm lever            | 120 m      |   |                                  |                 |
|     Sea thrust                | 8.07 MN    |   |                                  |                 |
|     Sea arm lever             | 5 m        |   |                                  |                 |

: Relevant parameters {#tbl-parameters}

::: {.callout-important title="Note: 8th October 2025"}
In the example in class, I gave $s = 130$. As you can see in the following calculations, this value for $s$ was incorrect. 

If I put 130 into the calculation instead of what's given below, the result for the vertical capacity is 101 MN. 

Apologies!
:::

# Results
## 1. VHM contributions
To get the vertical contribution, let's say $V_{demand}$, it's simply the sum of the weights of the nacelle, tower, concrete and the ballast.

However, there is also a buoyant weight acting on the volume of displaced water. This also needs to be included so that we have the *effective weight* of the structure. Therefore:

In [1]:
buoyant_contribution = (2965*10)/1000  # [MN]
V_demand = (4.1+3+30+38)-buoyant_contribution
print(f"The effective vertical load is {V_demand:.0f} MN, including the buoyancy contribution of {buoyant_contribution:.0f} MN")

The effective vertical load is 45 MN, including the buoyancy contribution of 30 MN


The horizontal component is simply just the sum of the loads imposed by both the wind and sea. These are normally distributed loads, although they have been converted into a representative point load at a certain point on the foundation/turbine.

In [2]:
H_demand = 2.03+8.07
print(f"The horizontal load is {H_demand:.0f} MN")

The horizontal load is 10 MN


The moment load is simply these horizontal loads, multiplied by the distance away from the foundation base (i.e. the lever arm). For this exercise, we assume they're just acting in the same direction, representing the worst possible case. 

In [3]:
M_demand = 2.03*120 + 8.07*5
print(f"The moment load is {M_demand:.0f} MNm")

The moment load is 284 MNm


## 2. Vertical stability
Since the foundation is directly on sand, our focus is on an effective stress analysis (assuming drained conditions). Therefore the following formula is applicable:

$$
q_{lim} = \frac{1}{2} \gamma' B_r N_\gamma K_\gamma + \sigma'_{v0}(N_q-1)K_q + c N_c s_c i_c d_c
$$ {#eq-qlim}

::: {.callout-note}
Importantly, **there is no surcharge load** since the foundation is founded directly on the seabed. Therefore $\sigma'_{v0}=0$, and likewise for the second term of the equation). 

Furthermore there is **no cohesion in sand** (or at least very, very little). Therefore $c=0$, and likewise for the third term of the equation.

:::

We've a circular foundation, so the first step then is to determine the effective width $B'$. This describes the reduced foundation area under combined horizontal/moment and vertical loading. For a circular foundation, the reduced area is semi-elliptical in-shape so for simplicity, ISO 19901-4 converts this shape to a rectangle with an equivalent area.

We've already been provided with $s$ in @tbl-parameters although I'll include my calculation of it here anyway.


In [4]:
import numpy as np

# Geometrical parameters
R = 23.5/2 # [m] Radius of the foundation
A = np.pi*R**2 # [m2]
e = M_demand/V_demand # Load eccentricity

s = (np.pi*R**2)/2 - (e*np.sqrt(R**2-e**2) + R**2 * np.arcsin(e/R))
L_eff = np.sqrt(2*s*np.sqrt((R+e)/(R-e)))
B_eff = L_eff * np.sqrt((R-e)/(R+e))
A_eff = B_eff*L_eff
print(f"The effective area is {A_eff:.0f} m2 (true area = {A:.0f}m2)")

The effective area is 155 m2 (true area = 434m2)


This is quite a large reduction in the soil/foundation contact zone! This is unsurprising to some extent: wind turbines are designed to catch as much horizontal load (wind) after all. But  if we were to go beyond this simple example, this potential conservatism may warrant further investigation into the prevailing wind and current patterns and into more complex numerical or physical models.

Next up are the bearing capacity factors $N_q$ and $N_\gamma$. The bearing capacity correction factors $K_q$, $K_\gamma$ have already been included in @tbl-parameters, although  but the calculation is included below.

::: {.callout-important}
Don't forget to include the model factor $\gamma_m$ any time a soil parameter is used! This accounts for any uncertainties we have in the soil properties, be it measurement uncertainty or inherent soil variability. 
:::

In [25]:
# Soil parameters
friction_angle = np.radians(32)
model_factor = 1.25 # gamma_m, to be applied to any soil properties
unit_weight = 20 # Saturated unit weight [kN/m3]
unit_weight_eff = 20-10 # Effective unit weight [kN/m3]
delta = np.radians(29) # Interface friction angle

# Bearing capacity factors
Nq = (np.tan(np.pi/4 + 0.5*np.arctan(np.tan(friction_angle)/model_factor))**2)*(np.exp(np.pi*(np.tan(friction_angle)/model_factor)))
print(f"Nq = {Nq:.2f}")

Ngamma = (3/2)*(Nq-1)*(np.tan(friction_angle)/model_factor)
print(f"Ngamma = {Ngamma:.2f}")

# Bearing capacity correction factors (Already provided in table)
igamma = 1-0.7*(H_demand/V_demand)**5
sgamma = 1-0.4*igamma*(B_eff/L_eff)
dgamma = 1
Kgamma = sgamma*dgamma*igamma
print(f"Kgamma = {Kgamma:.2f}")

Nq = 12.59
Ngamma = 8.69
Kgamma = 0.78


Finally, we put this all into @eq-qlim. Since $q_{lim}$ is not overly useful by itself, so we'll convert it to a force in kN. This is still done using the effective area. 

In [27]:
qlim = 0.5*unit_weight_eff*B_eff*Ngamma*Kgamma # kPa
V_capacity = (qlim*A_eff)/1000 # MN
print(f"The vertical capacity is {V_capacity:.0f} MN")

The vertical capacity is 48 MN


## 3. Resistance for sliding
The resistance to sliding is relatively straightforward to determine for a foundation with no embedment. To do this for sand, we take an effective stress analysis whereby:

$$
H_{ult} = V' \frac{\tan{\delta}}{\gamma_m}
$$


::: {.callout-important title="Correction 12/10/25"}
I incorrectly stated in the lecture notes that:

$$
H_{ult} = A_r (V' \tan{\delta})
$$

This has now been corrected in this example, resulting in a much lower (but still safe) horizontal capacity.

:::

In [35]:
H_capacity = (V_demand * np.tan(delta)/model_factor)
print(f"The horizontal capacity is {H_capacity:.0f} MN")

The horizontal capacity is 20 MN


:::{.callout-tip title="Bonus: Moment check" collapse="true"}
This wasn't requested in the initial brief, but as a quick check of the moments, we can compare the overturning moments (by wind and waves) to the restoring moments (self-weight of the structure) by:

$$
F h = V' D
$$
where $h$ is the distance from the pile base to the point upon which the force $F$ acts.

Therefore:


In [36]:
restoring_moment = V_demand * R
overturning_moments = 2.03*120 + 8.07*5
print(f"Restoring moment = {restoring_moment:.0f} MNm")
print(f"Overturning moment = {overturning_moments:.0f} MNm")
      

Restoring moment = 534 MNm
Overturning moment = 284 MNm


The restoring moment is greater than the overturning moment, so we're ok!
:::
<!-- #End of Quarto callout -->

## 4. Improvements to the design
Let's look at our unity checks:

In [37]:
print(f"V_demand/V_capacity = {V_demand/V_capacity:.2f}")
print(f"H_demand/H_capacity = {H_demand/H_capacity:.3f}")

V_demand/V_capacity = 0.94
H_demand/H_capacity = 0.501


Clearly, we have zero issues here with the horizontal capacity. Generally, drained conditions in sand don't pose an issue to horizontal stability. However, the horizontal capacity begins to become more critical in undrained scenarios. Of course, this will include foundation capacity in clay, however, undrained scenarios can even occur in sands under the short duration of a single wave load. Therefore both drained and undrained checks are recommended when performing a full foundation calculation. 

Even more importantly, the vertical capacity is *reallllllyyyy* close to the final design load. If we also take into account cyclic loading and corresponding degradation, we could well be approaching ULS failure. Surely this can't be the case?

Well there's a couple of things we're missing: 

* We don't have the full details of the loads provided. Have these been factored? Are they absolutely the worst-case scenario?
* We've also just assumed that the foundation rests on the seabed yet in reality, there the foundation is embedded underneath 3.5m of fill. 

If we want to improve the design, some foundation embedment may be required. And in reality, this is exactly what was done at Thornton Bank: the foundation was indeed embedded under both a fill and scour protection layer (@fig-thornton-bank-structure).

If we include embedment into our calculation, the effective stress is therefore no longer zero at the base of the foundation. This means the second (surcharge) term of @eq-qlim comes in to play. The first term remains the same. 


In [43]:
embedment = 3.5 # [m]
water_depth = 30
sig_eff = unit_weight_eff*embedment
print(f"Effective stress at foundation base = {sig_eff:.0f}kPa")

# Bearing capacity factor
Nq = (np.tan(np.pi/4 + 0.5*np.arctan(np.tan(friction_angle)/model_factor))**2) * np.exp(np.pi*(np.tan(friction_angle)/model_factor))
print(f"Nq = {Nq:.2f}")

iq = 1-0.5*(H_demand/V_demand)**5
sq = 1 + iq*(B_eff/L_eff) * np.sin(np.arctan(np.tan(friction_angle)/model_factor))
dq = 1 + 1.2*(R*2/B_eff) * (np.tan(friction_angle)/model_factor) * (1-np.sin(np.arctan(np.tan(friction_angle)/model_factor)))**2
Kq = sq*dq*iq
print(f"Kq = {Kq:.2f}")

qlim = 0.5*unit_weight_eff*B_eff*Ngamma*Kgamma + sig_eff*(Nq-1)*Kq # kPa
V_capacity_embedded = (qlim*A_eff)/1000 # MN
print()
print(f"The vertical capacity after embedment is {V_capacity_embedded:.0f} MN")
print(f"V_demand/V_capacity = {V_demand/V_capacity_embedded:.2f}")

Effective stress at foundation base = 35kPa
Nq = 12.59
Kq = 1.83

The vertical capacity after embedment is 163 MN
V_demand/V_capacity = 0.28


This is much better! This also gives "reserve capacity" when we start looking at long-term capacity, particularly under cyclic loading. 